In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Final Project - Lan/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Final Project - Lan


In [3]:
#Eymon Lan 661681496
#Final Project

import pandas as pd
import numpy as np
# Read the CSV file
df = pd.read_csv('Data.csv')

# Print the data
print(df.shape)

data = df.values
print(data.shape)

(284, 50)
(284, 50)


In [4]:
print(data)

[[0.00000000e+00 2.20000000e+01 0.00000000e+00 ... 2.65230000e+00
  0.00000000e+00 3.26470000e+00]
 [7.19999538e+03 2.29676103e+01 7.19999538e+03 ... 2.65230000e+00
  0.00000000e+00 3.26470000e+00]
 [7.20000000e+03 2.29676106e+01 7.20000000e+03 ... 2.65230000e+00
  0.00000000e+00 3.26470000e+00]
 ...
 [7.19999504e+03 3.05332738e+01 7.19999504e+03 ... 2.65230000e+00
  0.00000000e+00 3.26470000e+00]
 [7.20000000e+03 3.05332772e+01 7.20000000e+03 ... 2.65230000e+00
  0.00000000e+00 3.26470000e+00]
 [           nan            nan            nan ... 2.65230000e+00
  0.00000000e+00 3.26470000e+00]]


In [5]:
# filter out the rows and only keep final time step
rows_to_keep = np.where(data[:,0] == 7200)[0]
data1 = data[rows_to_keep, :]
print(data1.shape)

(100, 50)


In [6]:
# define input
y = np.linspace(1, 10, 100)
print(y.shape)
y=y.reshape((y.shape[0],1))
#print(len(x))
print(y.shape)
#print(y)

(100,)
(100, 1)


In [8]:
# Post processing data
Tvap = data1[:, 1:17:2]
Qtie = data1[:, 18:34:2]
Area = data1[:, 35:52:2]
Qflux = Qtie/Area
X = np.concatenate((Tvap,Qflux),axis=1)
#X = Tvap
print(Tvap.shape)
print(Qtie.shape)
print(X.shape)
print(X[99])
print(X)

(100, 8)
(100, 8)
(100, 16)
[30.53327724 34.21260102 37.02954878 39.32718303 41.52051328 43.79215637
 46.45094569 49.70131642  3.29769497  5.85123313  7.3871163   7.99548958
  8.30885681  5.79260658  3.38587989  1.46014067]
[[22.96761056 24.10617993 25.1118794  ...  0.18717071  0.07414814
   0.10632843]
 [23.05743763 24.22757811 25.25651934 ...  0.29003223  0.14083617
   0.13439286]
 [23.1477804  24.34845803 25.40070783 ...  0.39072832  0.20512395
   0.16112544]
 ...
 [30.40363916 34.04049214 36.8284584  ...  5.71594131  3.33675581
   1.44093444]
 [30.46860116 34.12655771 36.92915041 ...  5.75714462  3.36369509
   1.45057043]
 [30.53327724 34.21260102 37.02954878 ...  5.79260658  3.38587989
   1.46014067]]


In [9]:
Data_PP = np.concatenate((y,X),axis=1)
print(Data_PP.shape)

import pandas as pd

# mount your Google Drive to the Colab notebook


df = pd.DataFrame(Data_PP)

# write the DataFrame to a new Excel file in your Google Drive
with pd.ExcelWriter('/content/drive/My Drive/Colab Notebooks/Final Project - Lan/new_excel_file.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

(100, 17)


In [10]:
# ANN Model to predict heat flux
import tensorflow as tf
from sklearn.model_selection import train_test_split
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(70, 16)
(30, 16)
(70, 1)
(30, 1)


In [8]:
print(X_train.shape)
print(X_train[0])
print(y_train[0])

(70, 16)
[23.93660552 25.41125402 26.66404606 27.76573976 28.8647555  30.00970997
 31.34358857 32.98045941  0.77885025  1.37862467  1.74741693  1.84764319
  1.91238778  1.23158168  0.71852295  0.37403531]
[2.]


In [9]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(16,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Define your optimizer with a custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X, y, epochs=200, batch_size=15)

Streaming output truncated to the last 5000 lines.
7/7 [==============================] - 0s 7ms/step - loss: -724388347904.0000 - accuracy: 0.0100
Epoch 15008/20000
7/7 [==============================] - 0s 6ms/step - loss: -724609531904.0000 - accuracy: 0.0100
Epoch 15009/20000
7/7 [==============================] - 0s 7ms/step - loss: -724834975744.0000 - accuracy: 0.0100
Epoch 15010/20000
7/7 [==============================] - 0s 8ms/step - loss: -725043642368.0000 - accuracy: 0.0100
Epoch 15011/20000
7/7 [==============================] - 0s 9ms/step - loss: -725263056896.0000 - accuracy: 0.0100
Epoch 15012/20000
7/7 [==============================] - 0s 8ms/step - loss: -725486731264.0000 - accuracy: 0.0100
Epoch 15013/20000
7/7 [==============================] - 0s 8ms/step - loss: -725702934528.0000 - accuracy: 0.0100
Epoch 15014/20000
7/7 [==============================] - 0s 11ms/step - loss: -725923528704.0000 - accuracy: 0.0100
Epoch 15015/20000
7/7 [=======================

KeyboardInterrupt: ignored